## Load data

In [1]:
import pandas as pd

url_univclass= 'https://raw.githubusercontent.com/irenekarijadi/RF-LSTM-CEEMDAN/main/Dataset/data%20of%20UnivClass_Abby.csv'
univclass= pd.read_csv(url_univclass)
data_univclass= univclass[(univclass['timestamp'] > '2015-03-01') & (univclass['timestamp'] < '2015-06-01')]
dfs_univclass=data_univclass['energy']
datas_univclass=pd.DataFrame(dfs_univclass)


## import libraries

In [2]:
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter('ignore')

from PyEMD import CEEMDAN
import numpy
import math
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn import metrics

import time
import dataframe_image as dfi


## Import all functions from another notebook for building prediction methods

In [3]:
import Setting
from myfunctions import lr_model,svr_model,ann_model,rf_model,lstm_model,hybrid_ceemdan_rf,hybrid_ceemdan_lstm,proposed_method

## Import parameter settings

In [4]:
hours=Setting.n_hours
data_partition=Setting.data_partition
max_features=Setting.max_features
epoch=Setting.epoch
batch_size=Setting.batch_size
neuron=Setting.neuron
lr=Setting.lr
optimizer=Setting.optimizer

## Run the experiments
### Run this following cell will train and test the proposed method and other benchmark methods on University Classroom Dataset

In [11]:
#Linear Regression

start_time = time.time()
lr_univclass=lr_model(datas_univclass,hours,data_partition)
lr_time_univclass=time.time() - start_time
print("--- %s seconds - Linear Regression- univclass ---" % (lr_time_univclass))

#Support Vector Regression
start_time = time.time()
svr_univclass=svr_model(datas_univclass,hours,data_partition)
svr_time_univclass=time.time() - start_time
print("--- %s seconds - Support Vector Regression- univclass ---" % (svr_time_univclass))


#ANN
start_time = time.time()
ann_univclass=ann_model(datas_univclass,hours,data_partition)
ann_time_univclass=time.time() - start_time
print("--- %s seconds - ANN- univclass ---" % (ann_time_univclass))

#random forest
start_time = time.time()
rf_univclass=rf_model(datas_univclass,hours,data_partition,max_features)
rf_time_univclass=time.time() - start_time
print("--- %s seconds - Random Forest- univclass ---" % (rf_time_univclass))

#LSTM
start_time = time.time()
lstm_univclass=lstm_model(datas_univclass,hours,data_partition,max_features,epoch,batch_size,neuron,lr,optimizer)
lstm_time_univclass=time.time() - start_time
print("--- %s seconds - lstm- univclass ---" % (lstm_time_univclass))


#CEEMDAN RF
start_time = time.time()
ceemdan_rf_univclass=hybrid_ceemdan_rf(dfs_univclass,hours,data_partition,max_features)
ceemdan_rf_time_univclass=time.time() - start_time
print("--- %s seconds - ceemdan_rf- univclass ---" % (ceemdan_rf_time_univclass))

#CEEMDAN LSTM
start_time = time.time()
ceemdan_lstm_univclass=hybrid_ceemdan_lstm(dfs_univclass,hours,data_partition,max_features,epoch,batch_size,neuron,lr,optimizer)
ceemdan_lstm_time_univclass=time.time() - start_time
print("--- %s seconds - ceemdan_lstm- univclass ---" % (ceemdan_lstm_time_univclass))


#proposed method
start_time = time.time()
proposed_method_univclass=proposed_method(dfs_univclass,hours,data_partition,max_features,epoch,batch_size,neuron,lr,optimizer)
proposed_method_time_univclass=time.time() - start_time
print("--- %s seconds - proposed_method- univclass ---" % (proposed_method_time_univclass))



--- 0.07757997512817383 seconds - Linear Regression- univclass ---
--- 0.2982041835784912 seconds - Support Vector Regression- univclass ---
--- 1.9846718311309814 seconds - ANN- univclass ---
--- 1.0985586643218994 seconds - Random Forest- univclass ---
--- 7.163073778152466 seconds - lstm- univclass ---
--- 31.095949411392212 seconds - ceemdan_rf- univclass ---
--- 87.12494087219238 seconds - ceemdan_lstm- univclass ---
--- 81.20549058914185 seconds - proposed_method- univclass ---


## Summarize of experimental results with running time
### Run this following cell will summarize the result and generate output used in Section 4.4 (Table 3) for University Classroom dataset

In [12]:
running_time_univclass=pd.DataFrame([lr_time_univclass,svr_time_univclass,ann_time_univclass,
                                   rf_time_univclass,lstm_time_univclass,ceemdan_rf_time_univclass,
                                   ceemdan_lstm_time_univclass,proposed_method_time_univclass])
running_time_univclass=running_time_univclass.T
running_time_univclass.columns=['LR','SVR','ANN','RF','LSTM','CEEMDAN RF','CEEMDAN LSTM','Proposed Method']

proposed_method_univclass_df=proposed_method_univclass[0:3]
result_univclass=pd.DataFrame([lr_univclass,svr_univclass,ann_univclass,rf_univclass,lstm_univclass,ceemdan_rf_univclass,
                    ceemdan_lstm_univclass,proposed_method_univclass_df])
result_univclass=result_univclass.T
result_univclass.columns=['LR','SVR','ANN','RF','LSTM','CEEMDAN RF','CEEMDAN LSTM','Proposed Method']
univclass_summary=pd.concat([result_univclass,running_time_univclass],axis=0)

univclass_summary.set_axis(['MAPE(%)', 'RMSE','MAE','running time (s)'], axis='index')

univclass_summary.style.set_caption("University Classroom Results")

index = univclass_summary.index
index.name ="University Classroom results"
univclass_summary

,LR,SVR,ANN,RF,LSTM,CEEMDAN RF,CEEMDAN LSTM,Proposed Method
University Classroom results,,,,,,,,
MAPE(%),3.655593,5.807452,5.983385,6.777798,5.169115,3.084158,2.123226,1.964881
RMSE,1.628556,2.529214,2.401744,2.649659,2.169301,1.332275,0.866036,0.815484
MAE,1.061574,1.779553,1.748034,2.017364,1.530002,0.919540,0.613114,0.569825
running time (s),0.077580,0.298204,1.984672,1.098559,7.163074,31.095949,87.124941,81.205491


In [13]:
#export table to png
#dfi.export(univclass_summary,"univclass_summary_table.png")

## Calculate percentage improvement
### Run this following cell will calculate percentage improvement and generate output used in Section 4.4 (Table 4) for University Classroom dataset

In [14]:
pMAPE_LR_vs_Proposed_univclass=((lr_univclass[0]-proposed_method_univclass[0])/lr_univclass[0])*100
pRMSE_LR_vs_Proposed_univclass=((lr_univclass[1]-proposed_method_univclass[1])/lr_univclass[1])*100
pMAE_LR_vs_Proposed_univclass=((lr_univclass[2]-proposed_method_univclass[2])/lr_univclass[2])*100

pMAPE_SVR_vs_Proposed_univclass=((svr_univclass[0]-proposed_method_univclass[0])/svr_univclass[0])*100
pRMSE_SVR_vs_Proposed_univclass=((svr_univclass[1]-proposed_method_univclass[1])/svr_univclass[1])*100
pMAE_SVR_vs_Proposed_univclass=((svr_univclass[2]-proposed_method_univclass[2])/svr_univclass[2])*100

pMAPE_ANN_vs_Proposed_univclass=((ann_univclass[0]-proposed_method_univclass[0])/ann_univclass[0])*100
pRMSE_ANN_vs_Proposed_univclass=((ann_univclass[1]-proposed_method_univclass[1])/ann_univclass[1])*100
pMAE_ANN_vs_Proposed_univclass=((ann_univclass[2]-proposed_method_univclass[2])/ann_univclass[2])*100

pMAPE_RF_vs_Proposed_univclass=((rf_univclass[0]-proposed_method_univclass[0])/rf_univclass[0])*100
pRMSE_RF_vs_Proposed_univclass=((rf_univclass[1]-proposed_method_univclass[1])/rf_univclass[1])*100
pMAE_RF_vs_Proposed_univclass=((rf_univclass[2]-proposed_method_univclass[2])/rf_univclass[2])*100

pMAPE_LSTM_vs_Proposed_univclass=((lstm_univclass[0]-proposed_method_univclass[0])/lstm_univclass[0])*100
pRMSE_LSTM_vs_Proposed_univclass=((lstm_univclass[1]-proposed_method_univclass[1])/lstm_univclass[1])*100
pMAE_LSTM_vs_Proposed_univclass=((lstm_univclass[2]-proposed_method_univclass[2])/lstm_univclass[2])*100

pMAPE_ceemdan_rf_vs_Proposed_univclass=((ceemdan_rf_univclass[0]-proposed_method_univclass[0])/ceemdan_rf_univclass[0])*100
pRMSE_ceemdan_rf_vs_Proposed_univclass=((ceemdan_rf_univclass[1]-proposed_method_univclass[1])/ceemdan_rf_univclass[1])*100
pMAE_ceemdan_rf_vs_Proposed_univclass=((ceemdan_rf_univclass[2]-proposed_method_univclass[2])/ceemdan_rf_univclass[2])*100


pMAPE_ceemdan_lstm_vs_Proposed_univclass=((ceemdan_lstm_univclass[0]-proposed_method_univclass[0])/ceemdan_lstm_univclass[0])*100
pRMSE_ceemdan_lstm_vs_Proposed_univclass=((ceemdan_lstm_univclass[1]-proposed_method_univclass[1])/ceemdan_lstm_univclass[1])*100
pMAE_ceemdan_lstm_vs_Proposed_univclass=((ceemdan_lstm_univclass[2]-proposed_method_univclass[2])/ceemdan_lstm_univclass[2])*100


df_PI_univclass=[[pMAPE_LR_vs_Proposed_univclass,pMAPE_SVR_vs_Proposed_univclass,pMAPE_ANN_vs_Proposed_univclass,
                pMAPE_RF_vs_Proposed_univclass,pMAPE_LSTM_vs_Proposed_univclass,pMAPE_ceemdan_rf_vs_Proposed_univclass,
                pMAPE_ceemdan_lstm_vs_Proposed_univclass],
                [pRMSE_LR_vs_Proposed_univclass,pRMSE_SVR_vs_Proposed_univclass,pRMSE_ANN_vs_Proposed_univclass,
                pRMSE_RF_vs_Proposed_univclass,pRMSE_LSTM_vs_Proposed_univclass,pRMSE_ceemdan_rf_vs_Proposed_univclass,
                pRMSE_ceemdan_lstm_vs_Proposed_univclass],
                [pMAE_LR_vs_Proposed_univclass,pMAE_SVR_vs_Proposed_univclass,pMAE_ANN_vs_Proposed_univclass,
                pMAE_RF_vs_Proposed_univclass,pMAE_LSTM_vs_Proposed_univclass,pMAE_ceemdan_rf_vs_Proposed_univclass,
                pMAE_ceemdan_lstm_vs_Proposed_univclass]]

PI_univclass=pd.DataFrame(df_PI_univclass, columns=["LR vs. Proposed Method", "SVR vs. Proposed Method","ANN vs. Proposed Method",
                                      "RF vs. Proposed Method","LSTM vs. Proposed Method","CEEMDAN RF vs. Proposed Method",
                                      "CEEMDAN RF LSTM vs. Proposed Method"])
PI_univclass= PI_univclass.round(decimals = 2)
PI_univclass.set_axis(['MAPE(%)', 'RMSE','MAE'], axis='index')
PI_univclass.style.set_caption("Percentage Improvement-University Classroom Building")
index = PI_univclass.index
index.name = "Percentage Improvement university classroom"
PI_univclass

,LR vs. Proposed Method,SVR vs. Proposed Method,ANN vs. Proposed Method,RF vs. Proposed Method,LSTM vs. Proposed Method,CEEMDAN RF vs. Proposed Method,CEEMDAN RF LSTM vs. Proposed Method
Percentage Improvement university classroom,,,,,,,
MAPE(%),46.25,66.17,67.16,71.01,61.99,36.29,7.46
RMSE,49.93,67.76,66.05,69.22,62.41,38.79,5.84
MAE,46.32,67.98,67.40,71.75,62.76,38.03,7.06


In [15]:
#export table to png
#dfi.export(PI_univclass,"PI_univclass_table.png")